In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize

#### Load in and process

In [ ]:
df = pd.read_excel('data/save_file.xlsx')
print(df['compute'].isna().sum())
df.replace('',np.nan,inplace=True)
print(df['compute'].isna().sum())

##### Add columns and sort out 'poss 1e23/1e25'

# Reset the ones we've set
df.loc[~df["compute"].isna(), "poss1e23"] = np.nan
df.loc[~df["compute"].isna(), "poss1e25"] = np.nan

print(df['compute'].isna().sum())

# Set some temporary placeholder values
# TODO: revisit
# df.loc[(df["poss1e25"] == "checked"), "compute"] = 1.01e25  # placeholder
# df.loc[((df["poss1e23"] == "checked") & (df["poss1e25"] != "checked")), "compute"] = 1.01e23  # placeholder

# We want to handle these leading models manually via the above compute estimates.
assert df[(df["poss1e25"] == "checked") & (df["compute"].isna())].size == 0

# We sample 1e23-1e25 models with unknown compute from the existing empirical distribution.
# TODO: revisit
poss1e23 = ((df["poss1e23"] == "checked") & (df["poss1e25"] != "checked"))
df.loc[poss1e23, "compute"] = df[(df["compute"] >= 1e23) & (df["compute"] < 1e25)]["compute"].sample(poss1e23.sum(), random_state=0).values



##setting columns
df["date"] = pd.to_datetime(df["date"])

df['compute'] = pd.to_numeric(df['compute'],errors='coerce')
df["log_compute"] = np.log10(df["compute"])

df["date_float"] = df["date"].dt.year + df["date"].dt.month/12

df = df.sort_values("date")

df['year'] = df['date'].dt.year

In [ ]:
systems_to_remove = ['AlphaGo Zero', 'AlphaZero']
print(len(df))
df = df[~df['model'].isin(systems_to_remove)]
print(len(df))

#### Regression

In [ ]:
## regressions

from sklearn.linear_model import LinearRegression

#filtering
start_date = '2017-01-01'
end_date = '2024-01-01'
filtered_df = df[(df['date']>start_date) & (df['date']<end_date)]
filtered_df = filtered_df[~filtered_df['log_compute'].isna()] #remove Nan compute rows


#lin regress for means
year_grouped_df = filtered_df.groupby(['year'])
grouped_df_mean = year_grouped_df['log_compute'].mean()
aggregate_compute = year_grouped_df['compute'].sum()
log_aggregate_compute = np.log10(aggregate_compute)

X = grouped_df_mean.index.values


mean_log_compute_model = LinearRegression()
mean_log_compute_model.fit(X.reshape(-1,1),grouped_df_mean)

log_aggregate_compute_model = LinearRegression()
log_aggregate_compute_model.fit(X.reshape(-1,1),log_aggregate_compute)

del filtered_df #remove filtered df

#### Exponential counts

In [ ]:
## exp counts fit

from scipy.stats import t

def exp_pred_counts(years,year_counts,future_years,alpha=0.10):
    mapped_years = np.arange(0,len(year_counts)).astype('float')

    def exp_fit(x,a,b):
        return a*np.exp(b*x)
    
    popt,pcov = optimize.curve_fit(exp_fit,mapped_years.astype(float),year_counts.values.astype(float))
    pred_counts = exp_fit(future_years-years[0],*popt).astype(int) 

    #calculatiing confidence bounds
    #assuming log normal uncertainty, 90% CI
    pred_counts_fit = exp_fit(mapped_years,*popt)
    log_pred_counts_fit = np.log(pred_counts_fit)
    log_obs_counts = np.log(year_counts.values.astype(float))
    residuals = log_pred_counts_fit - log_obs_counts #we're calculating residuals of log counts 
    SEP = np.sqrt(np.sum(residuals**2)/(len(year_counts-2)))


    alpha = alpha #90% conf interval
    dof = len(year_counts) - 2 #apparenlty for linear function the dof is n-2
    crit_t_value = stats.t.ppf(1-alpha/2,dof)
    pred_delta = crit_t_value*SEP 

    years_all = np.concatenate([years,future_years])
    preds_all = np.concatenate([pred_counts_fit,pred_counts])
    log_pred_UB = np.log(preds_all)+pred_delta
    log_pred_LB = np.log(preds_all)-pred_delta
    pred_counts_UB = np.exp(log_pred_UB)
    pred_counts_LB = np.exp(log_pred_LB)

    return years_all,preds_all,pred_counts_UB,pred_counts_LB

#### Model distribution

In [ ]:
import seaborn as sns
from scipy.stats import gaussian_kde,norm,linregress
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import sys

In [ ]:
data_fit = None #var place holder
mu_0 = None
def trunc_norm_NLL(sigma): #NLL assuming we're sampling from  truncated normal 
    ll = norm.logpdf(data_fit.to_numpy(),mu_0,sigma) - np.log(1-norm.cdf(data_fit.to_numpy().min(),mu_0,sigma))
    return -np.sum(ll)

In [ ]:
## SCATTER PLOT - just to visualise

#filtering
start_date = '2017-01-01'
end_date = '2024-01-01'
filtered_df = df[(df['date']>start_date) & (df['date']<end_date)]



filtered_df['year'] = filtered_df['date'].dt.year
filtered_df = filtered_df[~filtered_df['log_compute'].isna()] #remove Nan compute rows

#lin regress for means
grouped_df_mean = filtered_df.groupby(['year'])['log_compute'].mean().reset_index()

X = grouped_df_mean['year'].values
mean_log_compute = grouped_df_mean['log_compute'].values

mean_log_compute_model = LinearRegression()
mean_log_compute_model.fit(X.reshape(-1,1),mean_log_compute)

#plot
fig_S,ax_S=plt.subplots()
#sns.regplot(data=filtered_df,x='date_float',y='log_compute',color='tab:blue')
sns.stripplot(x='year',y='log_compute',data=filtered_df,jitter=True,ax=ax_S)
sns.regplot(x=pd.Categorical(filtered_df['year']).codes,y='log_compute',data=filtered_df,scatter=False,ax=ax_S)
ymin,ymax = ax_S.get_yticks().min(),ax_S.get_yticks().max()
ax_S.set_yticks(np.arange(ymin,ymax,1))
#ax_S.axhline(y=23.7)

del filtered_df

In [ ]:
#### KDEs

#filtering
start_date = '2017-01-01'
filtered_df = df[df['date']>start_date]


PLOT=True
FIT_2024_RT = True
mean_2024 = mean_log_compute_model.predict(np.array(2024).reshape(-1,1))[0]


start_year=2017
end_year = 2024 

years = np.arange(start_year,end_year+1)
log_compute_min = 10
log_compute_max = 28
x = np.linspace(log_compute_min,log_compute_max,1000)


nrows = 4
ncols = 2
fig,axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=(8,10))
axs_ravel = np.ravel(axs)
MEAN,VAR= [],[]



for idx,year in enumerate(years):
    try:ax=axs_ravel[idx]
    except: pass


    year_filtered_df = filtered_df[filtered_df['year']==year]
    nan_frac =(year_filtered_df['log_compute'].isna().sum())/len(year_filtered_df)
    year_filtered_df = year_filtered_df[~year_filtered_df['log_compute'].isna()]
    log_compute_data = year_filtered_df['log_compute']

    if year==2024:
        #find sigma from right tail
        if FIT_2024_RT:
            RT_filtered_data = log_compute_data[log_compute_data>mean_2024]
            #print(len(RT_filtered_data)/len(log_compute_data)) #sanity check
            init_sigma = np.std(RT_filtered_data)
            data_fit = RT_filtered_data
            mu_0 = mean_2024
            sigma = (minimize(trunc_norm_NLL,[init_sigma])).x

            mean,sigma=mu_0,sigma #variable renaming
            MEAN.append(mean)
            VAR.append(sigma[0]**2)

    #find means in standard way
    else: 
        mean,std = np.mean(log_compute_data),np.std(log_compute_data)
        MEAN.append(mean); VAR.append(std**2)
        kde = gaussian_kde(log_compute_data)
        norm_pdf = norm.pdf(x,mean,std)


    sns.kdeplot(log_compute_data,fill=True,ax=ax,label='KDE')
    ax.plot(x,norm_pdf,label='norm fit')
    ax.set_title(f'{year},nan_frac={np.round(nan_frac,1)}')

fig.tight_layout()

'''
##predicted pdfs
CONST_VAR=True
tmp_year = 2021

if CONST_VAR:
    idx_tmp_year = np.where(years==tmp_year)[0][0]
    dist_var = np.mean(VAR[idx_tmp_year:])
else: 
    dist_var = None #not yet implemened

future_years = np.arange(2024,2028+1)
pred_means = mean_log_compute_model.predict(future_years.reshape(-1,1))

fig,axs=plt.subplots(nrows=3,ncols=2,figsize=(8,10))
axs_ravel = np.ravel(axs)
x_ = np.linspace(15,35)

for idx,year in enumerate(future_years):
    ax=axs_ravel[idx]
    norm_pdf = norm.pdf(x_,pred_means[idx],dist_var)
    ax.plot(norm_pdf)



fig.tight_layout()
'''

del filtered_df


In [ ]:
### Counting models above thresholds



CONST_VAR = True #take variance from year y onwards
tmp_year = 2021

years = np.arange(2017,2023+1)
future_years = np.arange(2024,2028+1)

#filtering
start_date = '2017-01-01'
end_date = '2024-01-01'
filtered_df = df[(df['date']>start_date) & (df['date']<end_date)]


year_counts = filtered_df['year'].value_counts().sort_index()
year_counts = year_counts.loc[2017:2023]

years_all,pred_counts,pred_counts_UB,pred_counts_LB = exp_pred_counts(years,year_counts,future_years)

if CONST_VAR:
    idx_tmp_year = np.where(years==tmp_year)[0][0]
    dist_var = np.mean(VAR[idx_tmp_year:])
    dist_var = 1.1
else: 
    dist_var = None #not yet implemened


threshold_count = 0
threshold = 25
stt_bin,stop_bin,num = 23,30,1000
x=np.linspace(start=stt_bin,stop=stop_bin,num=num)
bins = np.arange(stt_bin,stop_bin+1)

predicted_bin_counts_df = pd.DataFrame(index=bins[:-1],columns=future_years)


for idx,year in enumerate(future_years):
    fmt_year = np.array(year).reshape(-1,1)

    mean,sigma = mean_log_compute_model.predict(fmt_year),np.sqrt(dist_var)
    norm_pdf = norm.pdf(x,mean,sigma)
    norm_cdf = norm.cdf(bins,loc=mean,scale=sigma) #cdf at bin edges
    bin_pmfs = np.diff(norm_cdf)

    counts_idx = np.where(years_all==year)[0][0]
    model_count = pred_counts[counts_idx]

    freq_dist = (model_count*bin_pmfs).astype(int)

    predicted_bin_counts_df[year]=freq_dist


del filtered_df


#### Training compute budget verification

In [ ]:



aggregate_log_compute_pred = log_aggregate_compute_model.predict(future_years.reshape(-1,1))

method = 'bin_median'
bin_index = (predicted_bin_counts_df.index.values[:,None]).astype(float) #formating, float to stop overflow
if method=='bin_median':
    index_mult_factor = 10**(bin_index+0.5) #midpoint vals
if method=='bin_lb':
    index_mult_factor = 10**(bin_index+0)
    
distribution_aggregate_log_compute = np.log10((index_mult_factor*predicted_bin_counts_df).sum(axis=0))
#predicted_bin_counts_df.loc['Aggregate log flop'] = new_row


##plot
years = predicted_bin_counts_df.columns.values
dist_to_reg_ratio = 10**(distribution_aggregate_log_compute-aggregate_log_compute_pred)

plt.bar(years,dist_to_reg_ratio)
plt.title('Ratio of total compute (distribution v.s. raw extrapolation)')



mean_log_compute_pred = mean_log_compute_model.predict(future_years.reshape(-1,1))


#### Empirical training compute distributions

In [ ]:
import sys

start_date = '2017-01-01'
end_date = '2024-01-01'
filtered_df = df[(df['date']>start_date) & (df['date']<end_date)]
filtered_df = filtered_df[~filtered_df['compute'].isna()]

years = filtered_df.year.unique()

year_grouped_df = filtered_df.groupby(['year'])
total_compute = year_grouped_df['compute'].sum()
log_total_compute = np.log10(total_compute)
frontier_models = year_grouped_df['compute'].max()

cols = years
index = ['f0-1','f1-2','f2-3','other']
df_store = pd.DataFrame(columns=cols,index=index)

for year in years:
    year_tmp_df = filtered_df[filtered_df['year']==year]
    year_total_compute = year_tmp_df['compute'].sum()
    frontier_model = frontier_models[year]
    
    bins_ = (np.log10(frontier_model) - np.arange(0,len(index)))
    bins_ = np.append(bins_,-0.1)
    year_tmp_df['binned'] = pd.cut(df['log_compute'],bins=bins_[::-1],labels=index[::-1])
    total_compute_bins = year_tmp_df.groupby(['binned'])['compute'].sum()
    frac_compute = round(total_compute_bins/year_total_compute,2)
    df_store[year]=frac_compute

df_store = df_store.cumsum()

new_index = ['Within 1 OOM', 'Within 2 OOM','Within 3 OOM','All']
df_store.index=new_index
#del filtered_df